## Задача по курсу "Вычислительная линейная алгебра"

Реализовать $LU-$разложение (с выбором ведущего элемента), использовать его для решения уравнения $Ax=b$. Организовать проверку, вычислив $A\hat{x}-b$, где $\hat{x}$ - найденное решение.


In [7]:
import numpy as np  # the library for matrix calculations

In [8]:
n = 100
A = np.random.randn(n, n) * 10
b = np.random.randn(n) * 10

In [9]:
# lup decomposition

def lup_decompose(A):
    """ A - input matrix 
        
        return:
        P - permutation matrix ,s.t. PA = LU
        L - lower triangle matrix with 1 on the diagonal
        U - upper triangle matrix
        C - matrix with elements of U on and above the diagonal and with
            elements of L below the diagonal
    """
    n = A.shape[0]
    C = A.copy()
    P = np.eye(n)

    for i in range(n):
        pivotValue = np.max(np.abs(C[i:, i])) # value of the pivot element
        pivot = np.argmax(np.abs(C[i:, i])) + i # number of row, containing pivot element
        if pivotValue != 0:
            P[[pivot, i], :] = P[[i, pivot], :]  # permutation
            C[[pivot, i], :] = C[[i, pivot], :]  # permutation
        for j in range(i+1,n):
            C[j, i] = C[j, i] / C[i, i]    # a_{ji} = a_{ji} / a_{ii}
            for k in range(i+1,n):
                C[j, k] -= C[j, i] * C[i, k]  #a_{jk} = a_{jk} - a_{ji}a_{ik}
    
    l_mask = np.array([[1]*(i) + [0]*(n-i) for i in range(n)])
    L = C * l_mask + np.diag(np.ones(n))
    U = C - L +  np.diag(np.ones(n))

    return P, L, U, C

In [10]:
P, L, U, _ = lup_decompose(A)
np.allclose(P @ A - L @ U, np.zeros((n, n))) # checks if PA-LU is close to zero matrix

True

In [11]:
# calculates inverse matrix to L

def inv_lower(L):
    n = L.shape[0]
    Inv = np.eye(n)
    
    for i in range(n):
        Inv[i] /= L[i, i]
        for j in range(i + 1, n):
            Inv[j] -= Inv[i] * L[j, i]
    return Inv

In [12]:
# calculates inverse matrix to U

def inv_upper(U):
    n = U.shape[0]
    Inv = np.eye(n)
    
    for i in range(n)[::-1]:
        Inv[i] /= U[i, i]
        for j in range(i)[::-1]:
            Inv[j] -= Inv[i] * U[j, i]
    return Inv

In [13]:
# Ax = b => PAx = LUx = Pb
# returns x = U^{-1} L^{-1} (Pb)

def lup_solve(A, b):
    P, L, U, _ = lup_decompose(A)
    Pb = P @ b
    x = inv_upper(U) @ inv_lower(L) @ Pb
    
    return x

In [14]:
x = lup_solve(A, b)

In [15]:
np.allclose(A @ x - b, np.zeros(n)) # checks if Ax - b is close to zero vector

True

In [16]:
np.linalg.norm(A @ x - b, 2) # calculates ||Ax - b||_2 ^2

3.233978876752354e-11

In [17]:
import scipy.linalg

In [18]:
P, L, U = scipy.linalg.lu(A)                          # A = PLU, where P - inverse permutation matrix !
x = inv_upper(U) @ inv_lower(L) @ P.transpose() @ b   #P^T = P^{-1} => x = U^{-1} L^{-1} P^T b
np.linalg.norm(A @ x - b, 2)                          # calculates ||Ax - b||_2 ^2

3.2647165327946756e-11

In [19]:
# calculation of the maximum eigenvalue of the matrix

def max_eigval(A, x=None, eps=1e-14):
    if x is None:
        x = np.random.randn(A.shape[0])
    lam = 1
    dif = 1
    while (dif > eps):
        lam_prev = lam
        y = A @ x
        x = y / np.linalg.norm(y, 2)
        lam = (x @ A @ x).item()
        dif = np.abs(lam - lam_prev)
    return lam

In [20]:
# calculation of the condition number of the matrix

def cond(A):
    P, L, U, _ = lup_decompose(A)
    PA_inv = inv_upper(U) @ inv_lower(L)
    PA = P @ A
    PA_norm = np.sqrt(max_eigval((PA.transpose() @ PA)))
    PA_inv_norm = np.sqrt(max_eigval(PA_inv.transpose() @ PA_inv))
    cond = PA_norm * PA_inv_norm
    
    return cond

In [21]:
# comparison of the condition numbers calculated by the inbuilt method and by one implemented here.

print(cond(A))
print(np.linalg.cond(A))

3657.0637720955397
3657.0637720949644
